# Import libraries

In [1]:
import random
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from data_loader import JesterV1Dataset

# Load data

### Define arguments

In [2]:
# Define metadata
batch_size = 8

In [11]:
# Define data transforms
data_transforms = transforms.Compose([
    transforms.Resize((112, 112)),
    #transforms.ToTensor(),
    #transforms.Normalize(
    #    mean = [0.485, 0.456, 0.406],
    #    std = [0.229, 0.224, 0.225]
    #), # --> Subtracts the mean values and then divides by std for each color channel
])

In [12]:
# Create dataset instance
train_dataset = JesterV1Dataset(
    root_dir = '../datasets/JESTER-V1',
    annotation_file = 'jester-v1-train.txt',
    num_frames = 30,
    transform = data_transforms
)

print(train_dataset.__len__())

118562


In [13]:
# Create a DataLoader instance
data_loader = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle = False
)

### Double check

In [14]:
count = -1
batch_idx = random.randint(0, train_dataset.__len__() // batch_size)
# Iterate through the DataLoader to get one batch
for batch in data_loader:
    # Random batch
    count += 1
    if count != batch_idx:
        continue
    
    # Get inputs and labels
    inputs, labels = batch
    print(inputs.shape, labels.shape)
    # Random sample
    sample_idx = random.randint(0, batch_size - 1)
    input_sample, label_sample = inputs[sample_idx], labels[sample_idx]
    print(input_sample.shape, label_sample.shape)
    # Convert input_sample to numpy and transpose (if needed)
    input_sample = input_sample.numpy()
    input_sample = input_sample.transpose((1, 2, 0)) # (C, H, W) --> (H, W, C)
    
    # Break after get a random batch
    break

TypeError: Unexpected type <class 'numpy.ndarray'>